In [2]:
from transformers import BitsAndBytesConfig

from torch import bfloat16

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, #4-bit quantization
    bnb_4bit_quant_type='nf4', #Normalized float 4
    bnb_4bit_use_double_quant=True, #Second quantization after the first
    bnb_4bit_compute_dtype=bfloat16 #Computation type
)

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map='auto'
)

pipe = pipeline(model=model, tokenizer=tokenizer, task='text-generation')

RuntimeError: No GPU found. A GPU is needed for quantization.